# Arm and Gripper

In the previous tutorial, we explored the concept of *moves* and basic control. Now, let's delve deeper into what we can do with Reachy's arms and grippers.

## Initialize Your Robot

First, connect to your robot:

In [ ]:
from reachy2_sdk import ReachySDK
import time

reachy = ReachySDK(host='localhost')  # Replace with the actual IP

Next, we need to turn on the parts we want to use:

In [ ]:
reachy.l_arm.turn_on()
reachy.r_arm.turn_on()

Since the grippers are part of the arms, they will also be turned on. You could, of course, turn on the whole robot by calling `reachy.turn_on()` directly.
Let's check if the arms are on:

In [ ]:
print(reachy.r_arm.is_on())
print(reachy.l_arm.is_on())

## Check Arm Joints

Reachy's arm offers 7 degrees of freedom. It also gives access to one joint for the gripper.
The arm is divided as follow:
- shoulder, composed of 2 joints (pitch and roll)
- elbow, composed of 2 joints (yaw and pitch)
- wrist, composed of 3 joints (roll, pitch and yaw)

We refer to the shoulder, elbow and wrist as actuators.
For some actions, such as changing the compliancy, is the the lowest level of control you will have.

You can inspect the details of the arm with:

In [ ]:
reachy.r_arm.joints

In [ ]:
reachy.l_arm.joints

You can easily access the position of each joint in one call with `get_joint_positions()`

In [ ]:
reachy.r_arm.get_joints_positions()

In [ ]:
reachy.l_arm.get_joints_positions()

## Move the Arms

The simplest way to move an arm is to set the angle of each joint. Define a joint positions list:

In [ ]:
elbow_at_90_deg = [0, 0, 0, -90, 0, 0, 0] # only the elbow is set

Send the joint goal positions to the arm with `goto_joints()`

In [ ]:
reachy.r_arm.goto_joints(elbow_at_90_deg)
reachy.l_arm.goto_joints(elbow_at_90_deg)
while reachy.r_arm.get_move_playing().id != -1:
    time.sleep(0.5)

In order to easily move a single joint, we can use the `goto()` method on a given joint:

In [ ]:
reachy.r_arm.elbow.pitch.goto(0)

while reachy.r_arm.get_move_playing().id != -1:
    time.sleep(0.5)

This method works exactly like a move on the part. All gotos sent to joints are stacked on the part they belong to.
For example:

In [ ]:
part_move_id = reachy.r_arm.goto_joints([-10, -15, -15, -90, 0, 0, 0])
joint_move_id = reachy.r_arm.elbow.pitch.goto(0)

print(part_move_id)
print(joint_move_id)

print(reachy.r_arm.get_moves_queue())

`reachy.r_arm.elbow.pitch.goto(0)` command is stacked in r_arm moves queue.
The move on the elbow will be played when the previous command sent using `goto_joints()` will be over.

The arms should have moved in a way similar to what we saw in the [move tutorial](moves_introduction.ipynb). You already know that you can specify the duration or the interpolation mode of this kind of movement.

We've only seen movements expressed in the 'joint space', i.e., defined by a set of angles. How can we know the position of the gripper in space, or how can we reach an object for which we know its position? That's where kinematics come in.

## Kinematics

The kinematic model describes the motion of a robot in mathematical form without considering the forces and torque affecting it. It only focuses on the geometric relationship between elements.

We have defined the whole kinematic model of the arm. This means the translation and rotation required to go from one joint to the next one. 

[Long story](https://docs.pollen-robotics.com/sdk/first-moves/kinematics/) short, there are two types of kinematics:
- Forward kinematics: from the joints position, the position in space of the gripper is computed
- Inverse kinematics: from a given position of the gripper to reach, all joints positions are computed

### Forward Kinematics

Each arm has a `forward_kinematics` method that computes a 4x4 pose matrix (position and orientation of the gripper in space). For instance, the previous movement left the left elbow at 90°. The position of the gripper is:

In [ ]:
reachy.l_arm.forward_kinematics()

It is not mandatory to move the arm to compute forward kinematics. This can be done for any set of joint positions such as:

In [ ]:
reachy.l_arm.forward_kinematics([10, 0, 0, -90, 0, 0, 0])

Reachy didn't move, but you know where it would have gone with an additional 10° applied to the shoulder joint.

### Inverse Kinematics

Inverse kinematics works in the opposite way. Let's say you want to reach an object for which you know its position. What would be the set of joint positions to provide to `goto_joints`?

In [ ]:
import numpy as np
target = np.identity(4)
target[0][3] = 0.4 # x
target[1][3] = 0.1 # y
target[2][3] = -0.3 # z
target

In [ ]:
joints_positions = reachy.l_arm.inverse_kinematics(target)
joints_positions

In [ ]:
reachy.l_arm.goto_joints(joints_positions)

In [ ]:
reachy.set_pose("default")
reachy.l_arm.goto_from_matrix(target)

> All these moves are illustrated in [draw_square](draw_square.py). Check it out to see how to make Reachy draw a square with its right arm!

## Gripper Control

Finally, you may want to open or close a gripper to grab an object! Use the `close` or `open` method to do so:

In [ ]:
reachy.l_arm.gripper.close()
time.sleep(1)
reachy.l_arm.gripper.open()

Of course, you can partially open the gripper, and get its current state:

In [ ]:
reachy.l_arm.gripper.set_opening(55)
time.sleep(1)
reachy.l_arm.gripper.opening

# Set robot back to compliant mode

In [ ]:
reachy.set_pose('default', duration=5)
reachy.set_pose('default')
while reachy.l_arm.get_move_playing().id != -1 or reachy.r_arm.get_move_playing().id != -1 or reachy.head.get_move_playing().id != -1:
    time.sleep(0.5)
    
reachy.turn_off_smoothly()